In [1]:
"""
<메가박스 영화 데이터 전체 중 10개만 수집하기>
"""

'\n<메가박스 영화 데이터 전체 중 10개만 수집하기>\n'

In [2]:
### 라이브러리
import pandas as pd

### 날짜 라이브러리
from datetime import datetime

### 웹 데이터수집을 위해 직접 만든 웹컨트롤 라이브러리
from user_lib.webcontrol import WebControl

import time

In [3]:
### WebControl 클래스 생성하기
# - 크롬 브라우저가 오픈됩니다.
wc = WebControl()
wc

In [4]:
### 최초 접근 URL 지정하여 접근합니다.
wc.setUrlPageControl("https://www.megabox.co.kr/movie")

In [5]:
### 페이지 하단의 [더보기] 버튼이 나오지 않을 때까지 클릭이벤트 발생시키기
# - 모두 펼처 놓은 상태에서 영화 정보 일괄 수집

# - 더 보기 버튼 태그 위치 찾기
more_view_path = "#btnAddMovie"
more_view_element = wc.getElement(more_view_path)
# print(type(more_view_element), f"dd = {more_view_element}")

wc.setMoreClickEvent(more_view_element)

In [6]:
print(f"더보기 총 횟수 : {wc.more_view_cnt}")

더보기 총 횟수 : 5


In [7]:
### 현재 페이지의 영화 목록 데이터 수집하기
# - 각 영화 목록 중 클릭이 가능한(상세보기로 접근하기 위해) 태그의 위치 찾기
# - #movieList > li:nth-child(1) > div.movie-list-info > div.movie-score > a

# - 모든 영화 목록 정보를 한번에 수집할 수 있는 공통된 태그로 정리하기
#   --> li:nth-child(1)에서 li만 남기고 삭제
# movie_list_path = "#movieList > li > div.movie-list-info > div.movie-score > a"
movie_list_path = "div.movie-list-info"
movie_elements = wc.getElements(movie_list_path)

print(f"영화목록 갯수 = {len(movie_elements)} / type : {type(movie_elements)}")

영화목록 갯수 = 104 / type : <class 'list'>


In [8]:
"""
영화목록 125개를 모두 받아오기 위해서 반복 수행합니다.
 - 다만, 너무 많기에 5번만 반복 수행 해보도록 하겠습니다.
 - 수집 데이터명 : 기준년월일, 영화제목, 예매순위, 예매율, 
                  누적관객수, 실관람평점, 실관람평내용
"""
### 수집된 모든 웹 데이터를 데이터프레임에 담기위한 리스트 변수
movie_list = []

### 긍정/부정 데이터를 데이터프레임에 담기위한 리스트 변수
movie_review_list = []

### [기준년월일]을 담을 리스트 변수 정의 : 오늘 기준으로 자동 생성
ymd = str(datetime.today().strftime("%Y-%m-%d %H:%M:%S"))

### 전체 반복 수행하기
############ [임시로 10번만] 반복해 보기위해 count 변수 정의
cnt = 0
for m_element in movie_elements :
    cnt += 1    
    if cnt == 11 :
        break
    print(f"현재 반복횟수 : [{cnt}]")
        
    ### 각 영화 목록별 클릭하여 상세보기 페이지로 전환시키기
    wc.setClickEvent(m_element)
    
    ### 영화 한편에 대한 정보를 딕셔너리에 담기위한 딕셔너리 변수
    movie_dict = {}
    
    ### [기준년월일]을 딕셔너리에 담기
    movie_dict["기준년월일"] = ymd
    
    ### [영화제목]을 딕셔너리에 담기
    movie_dict["영화제목"] = wc.getElementText("div.movie-detail-cont > p.title")
        
    ### [예매순위]를 딕셔너리에 담기
    movie_dict["예매순위"] = wc.getElementText("div.rate > p.cont > em")
       
    ### [예매율]을 딕셔너리에 담기
    movie_dict["예매율"] = wc.getElementText("#rkTag")
    
    ### [누적관객수]를 딕셔너리에 담기
    movie_dict["누적관객수"] = wc.getElementText("#contents > div.movie-detail-page > div.movie-detail-cont > div.info > div.audience > p > em")
    
    ### [총실관람평점]을 딕셔너리에 담기
    movie_dict["총실관람평점"] = wc.getElementText("#subMegaScore > div.score.equal > div > div > em")
    
    ############ [각 dict 데이터를 list 추가하기] ############
    ### 영화 한편 당 정보를 담은 딕션러리를 리스트에 추가하기
    movie_list.append(movie_dict)
    
    # ------------------------------------------------
    
    ######### [영화 리뷰 긍정/부정 감성분석에 사용할 데이터 생성] ############
    ######### - 별도의 [데이터프레임]으로 구성하기 #########################
    ######### - 생성할 컬럼명 : 영화제목, 개별평점, 개별평내용, 구분(긍정/부정)
    ######### - 긍정/부정 계산 방법 : 평점의 점수가 5이상 이면 "1(긍정)"
    #########                      : 평점의 점수가 5미만 이면 "0(부정)"

    ### 위쪽 for문 밖에 새로운 리스트변수 및 데이터프레임 변수 생성하기
    # - 변수명 : movie_review_list 

    #------------[개별 댓글 -> 개별실관람평점 및 개별실관람평내용] 탭 이동 후 진행---------
    ### [실관람평 탭]으로 이동 클릭에 따른 페이지 전환 처리
    
    # - 실관람평탭 링크 처리하기
    review_tab_path = "#contentData > div:nth-child(5) > div.tab-list.fixed > ul > li:nth-child(2) > a"
    review_tab_element = wc.getElement(review_tab_path)
    
    # - 페이지 이동하기
    wc.setClickEvent(review_tab_element)
    
    ### 실관람평탭 페이지는 내부적으로 전환 처리되는 방식으로 
    # - 전환 후의 페이지 URL을 이용하여 직접 재접속하여 아래 실행하여야 함
    wc.setUrlPageControl(wc.getCurrentUrl())
    time.sleep(1)

    ### 현재 페이지에 있는 전체 [평점 조회]
    review_score_elements = wc.getElements("div.story-point > span")

    ### 현재 페이지에 있는 전체 [리뷰 내용 조회]
    review_content_elements = wc.getElements("div.story-txt")
    
    print(f"리뷰 평점 갯수 = {len(review_score_elements)} / type : {type(review_score_elements)}")
    print(f"리뷰 내용 갯수 = {len(review_content_elements)} / type : {type(review_content_elements)}")
    
    ### 전체 평점 조회 건수가 없으면.....................
    #   -> 수집할 데이터가 없다는 의미임
    #   -> 따라서, 모든 딕션너리 값들은 0 또는 ""을 입력 
    if len(review_score_elements) == 0 :
        ### 아래 쪽 for문 밖에 아래 정보를 담을 딕셔너리 변수 정의
        movie_review_dict = {}
        
        ### [영화제목]을 딕셔너리에 담기
        movie_review_dict["영화제목"] = movie_dict["영화제목"]
        
        ### [개별실관람평점]을 딕셔너리에 담기
        movie_review_dict["개별실관람평점"] = 0
    
        ### [개별실관람평내용]을 딕셔너리에 담기
        movie_review_dict["개별실관람평내용"] = ""
    
        ### [구분] 긍정/부정을 딕셔너리에 담기
        movie_review_dict["구분"] = ""
    
        ############ [각 dict 데이터를 list에 추가하기] ############
        ### 감성분석에 사용할 데이터 딕셔너리를 리스트에 추가하기
        movie_review_list.append(movie_review_dict)

    ### 전체 평점 조회 건수가 있으면.....................
    # - [평점 갯수]만큼 for문을 이용하여 반복
    # - 아래 내용을 반복하면서 리스트에 담기..
    else :         
        for i in range(0, len(review_score_elements), 1) :
            ### 아래 쪽 for문 밖에 아래 정보를 담을 딕셔너리 변수 정의
            movie_review_dict = {}
            
            ### [영화제목]을 딕셔너리에 담기
            movie_review_dict["영화제목"] = movie_dict["영화제목"]
            
            ### [개별실관람평점]을 딕셔너리에 담기
            movie_review_dict["개별실관람평점"] = review_score_elements[i].text
        
            ### [개별실관람평내용]을 딕셔너리에 담기
            movie_review_dict["개별실관람평내용"] = review_content_elements[i].text
        
            ### [구분] 긍정(1)/부정(1)을 딕셔너리에 담기
            # 긍정(1) : 평점이 6이상으로 | 부정(0) : 평점이 5이하로 임의 정의
            level = 0
            if int(review_score_elements[i].text) >= 6 :
                level = 1
            movie_review_dict["구분"] = level
        
            ############ [각 dict 데이터를 list 추가하기] ############
            ### 감성분석에 사용할 데이터 딕셔너리를 리스트에 추가하기
            movie_review_list.append(movie_review_dict)

    ################## 다시 목록으로 나가기(목록 리스트)
    wc.setBeforePage(-1)

print("웹 데이터 수집을 정상적으로 완료하였습니다.!!")

### 크롬 브라우저 닫기(종료)
wc.closeDriver()
print("크롬브라우저를 정상적으로 종료하였습니다.!!")

현재 반복횟수 : [1]
리뷰 평점 갯수 = 10 / type : <class 'list'>
리뷰 내용 갯수 = 10 / type : <class 'list'>
현재 반복횟수 : [2]
리뷰 평점 갯수 = 0 / type : <class 'list'>
리뷰 내용 갯수 = 0 / type : <class 'list'>
현재 반복횟수 : [3]
리뷰 평점 갯수 = 0 / type : <class 'list'>
리뷰 내용 갯수 = 0 / type : <class 'list'>
현재 반복횟수 : [4]
리뷰 평점 갯수 = 0 / type : <class 'list'>
리뷰 내용 갯수 = 0 / type : <class 'list'>
현재 반복횟수 : [5]
리뷰 평점 갯수 = 10 / type : <class 'list'>
리뷰 내용 갯수 = 10 / type : <class 'list'>
현재 반복횟수 : [6]
리뷰 평점 갯수 = 10 / type : <class 'list'>
리뷰 내용 갯수 = 10 / type : <class 'list'>
현재 반복횟수 : [7]
리뷰 평점 갯수 = 10 / type : <class 'list'>
리뷰 내용 갯수 = 10 / type : <class 'list'>
현재 반복횟수 : [8]
리뷰 평점 갯수 = 10 / type : <class 'list'>
리뷰 내용 갯수 = 10 / type : <class 'list'>
현재 반복횟수 : [9]
리뷰 평점 갯수 = 10 / type : <class 'list'>
리뷰 내용 갯수 = 10 / type : <class 'list'>
현재 반복횟수 : [10]
리뷰 평점 갯수 = 10 / type : <class 'list'>
리뷰 내용 갯수 = 10 / type : <class 'list'>
웹 데이터 수집을 정상적으로 완료하였습니다.!!
크롬브라우저를 정상적으로 종료하였습니다.!!


In [9]:
### 영화정보 담기
df_movie_info = pd.DataFrame(movie_list)

### 영화정보 보기
df_movie_info

,기준년월일,영화제목,예매순위,예매율,누적관객수,총실관람평점
0,2024-06-13 23:55:16,인사이드 아웃 2,1,60.9%,"196,508",9.2
1,2024-06-13 23:55:16,탈주,2,0%,0,
2,2024-06-13 23:55:16,하이재킹,3,0%,461,
3,2024-06-13 23:55:16,몽키맨,4,0%,0,
4,2024-06-13 23:55:16,극장판 하이큐!! 쓰레기장의 결전,5,3.3%,"632,696",9.1
5,2024-06-13 23:55:16,존 오브 인터레스트,6,2.4%,"77,608",8.7
6,2024-06-13 23:55:16,드라이브,7,2.1%,"20,082",7.6
7,2024-06-13 23:55:16,퓨리오사: 매드맥스 사가,8,1.9%,"1,459,854",9.2
8,2024-06-13 23:55:16,기괴도,9,1.6%,"1,937",6.5
9,2024-06-13 23:55:16,생츄어리,10,1.2%,"1,925",8


In [10]:
### 영화 사용자 리뷰 담기
df_movie_review = pd.DataFrame(movie_review_list)

### 영화 사용자 리뷰 보기
df_movie_review

,영화제목,개별실관람평점,개별실관람평내용,구분
0,인사이드 아웃 2,6,"성공하면 불안이, 실패하면 부리부리",1
1,인사이드 아웃 2,10,혹시나 1편보다 재미없는건 아닐까 걱정했는데 눈물 한바가지.,1
2,인사이드 아웃 2,10,사춘기자녀와 함께 보세요~ 딸과 엄마가 공감하는 이야기,1
3,인사이드 아웃 2,9,라일라 가스라이팅 극복기,1
4,인사이드 아웃 2,10,재밌었어요 잘봤습니다.,1
...,...,...,...,...
68,생츄어리,10,"""생명"" 이란 단어에 많은 생각이 드네요. 잘 봤습니다",1
69,생츄어리,10,동물원생추어리 영화가 아님.다큐!,1
70,생츄어리,8,생츄어리 생츄어리 생츄어리,1
71,생츄어리,8,재미나게 잘 보았습니다.,1


In [11]:
### 데이터프레임 생성 방법 샘플
# temp = [{"k1":1, "k2":2, "k3":3}, {"k1":4, "k2":5, "k3":6}]
# 또는 # temp = [("k1":1, "k2":2, "k3":3), ("k1":4, "k2":5, "k3":6)]
# df = pd.DataFrame(temp)
# df

In [12]:
### 수집된 웹 데이터의 가공 형태
# - 이 데이터를 데이터프레임으로 변환하였음
movie_list
movie_review_list

[{'영화제목': '인사이드 아웃 2',
  '개별실관람평점': '6',
  '개별실관람평내용': '성공하면 불안이, 실패하면 부리부리',
  '구분': 1},
 {'영화제목': '인사이드 아웃 2',
  '개별실관람평점': '10',
  '개별실관람평내용': '혹시나 1편보다 재미없는건 아닐까 걱정했는데 눈물 한바가지.',
  '구분': 1},
 {'영화제목': '인사이드 아웃 2',
  '개별실관람평점': '10',
  '개별실관람평내용': '사춘기자녀와 함께 보세요~ 딸과 엄마가 공감하는 이야기',
  '구분': 1},
 {'영화제목': '인사이드 아웃 2', '개별실관람평점': '9', '개별실관람평내용': '라일라 가스라이팅 극복기', '구분': 1},
 {'영화제목': '인사이드 아웃 2', '개별실관람평점': '10', '개별실관람평내용': '재밌었어요 잘봤습니다.', '구분': 1},
 {'영화제목': '인사이드 아웃 2',
  '개별실관람평점': '10',
  '개별실관람평내용': '즐거운 시간이 됬어요. 애들도 좋아해요',
  '구분': 1},
 {'영화제목': '인사이드 아웃 2',
  '개별실관람평점': '10',
  '개별실관람평내용': '1이 정말 인생영화라서 기대하고 봤는데 기대이상의 퀄리티와 내용이었어요 라일리의 그간 성장과정을 보면서 같이 자란 기분이라 너무 묘했고 감정선이 전부 이해 가서 재밌게 봤어요 강추',
  '구분': 1},
 {'영화제목': '인사이드 아웃 2',
  '개별실관람평점': '10',
  '개별실관람평내용': '캐릭터 너무 귀여워요 재밌어요 ㅎㅎㅎ',
  '구분': 1},
 {'영화제목': '인사이드 아웃 2',
  '개별실관람평점': '10',
  '개별실관람평내용': '돌비로 봐야하는 영화 재밌음',
  '구분': 1},
 {'영화제목': '인사이드 아웃 2',
  '개별실관람평점': '10',
  '개별실관람평내용': '나를 가장 나답게, 감정에 충실하게.',
  '구분': 1},
 {'영화제목': '